# Desafio Fictício - Aonde instalar pontos de iluminação pública para melhorar a segurança?

Apresente uma análise com visualização de dados, utilizando Python ou R e ferramentas para visualização de dados para responder à seguinte necessidade:

Considere que a Prefeitura de São Paulo disponha de orçamento para investir na instalação de 500 pontos de iluminação pública.
Quais pontos deveriam ser priorizados para melhorar a segurança pública?

Para tanto, utilize-se dos **dados de atendimento ao cidadão**
http://dados.prefeitura.sp.gov.br/dataset/dados-do-sp156, dos **dados de iluminação pública** http://dados.prefeitura.sp.gov.br/dataset/iluminacao-publica, dos **dados de segurança pública estaduais** http://catalogo.governoaberto.sp.gov.br/dataset/45-infocrim-informacoes-criminais e outros dados que julgar relevantes.

In [81]:
import pandas as pd
import numpy as np
import requests
import matplotlib as plt
import os

In [62]:
def get_public_data_sp(id):
    base_url = 'http://dados.prefeitura.sp.gov.br/api/action/datastore_search?resource_id={id}'
    url = base_url.format(id=id)
    print('Getting data from ', url)
    result = requests.get(url)
    dict_result = result.json()
    return dict_result['result']['records']

In [63]:
# Dados de atendimento ao cidadão

atend_base_url = 'http://dados.prefeitura.sp.gov.br/api/action/datastore_search?resource_id={id}'
atend_data_url = {}
atend_data_url['2017-07-01'] = 'c439343b-6e2e-4cc5-84fe-aba0e54688a1'
atend_data_url['2017-01-01'] = '542c3405-5a2d-4e76-b318-92ab4869d453'
atend_data_url['2016-07-01'] = 'c3ef2030-77ec-4eed-945b-297fd52459d1'
atend_data_url['2016-01-01'] = '3a5ff4b6-d6b3-458c-a050-04af05a171ee'

In [64]:
df_atend = []

for k, v in atend_data_url.items():
    df = pd.DataFrame(get_public_data_sp(v))
    df_atend.append(df)

Getting data from  http://dados.prefeitura.sp.gov.br/api/action/datastore_search?resource_id=c439343b-6e2e-4cc5-84fe-aba0e54688a1
Getting data from  http://dados.prefeitura.sp.gov.br/api/action/datastore_search?resource_id=542c3405-5a2d-4e76-b318-92ab4869d453
Getting data from  http://dados.prefeitura.sp.gov.br/api/action/datastore_search?resource_id=c3ef2030-77ec-4eed-945b-297fd52459d1
Getting data from  http://dados.prefeitura.sp.gov.br/api/action/datastore_search?resource_id=3a5ff4b6-d6b3-458c-a050-04af05a171ee


In [65]:
df_atends = pd.concat(df_atend)
df_atends.head()

,Assunto,Canal Atendimento,Data Abertura,Data Parecer,Distrito,Hora Abertura,Hora Parecer,Prefeitura Regional,Servico,Status Solicitaзгo,Status Solicitaусo,Status Solicitaчуo,Status Solicitaחדo,Tema,_id,Мrgсo,Уrgгo,гrgуo,׃rgדo
0,Remoзгo de grandes objetos,CENTRAL TELEFФNICA,2017-07-07T00:00:00,None,SACOMA,2017-11-01T12:48:00,None,IPIRANGA,Remoзгo de grandes objetos em vias pъblicas,AGUARDANDO ATENDIMENTO,NaN,NaN,NaN,Lixo e limpeza,1,NaN,AMLURB,NaN,NaN
1,Linhas e itinerбrios de фnibus,CENTRAL TELEFФNICA,2017-07-07T00:00:00,2017-07-17T00:00:00,JARDIM ANGELA,2017-11-01T12:48:00,2017-11-01T18:50:00,M'BOI MIRIM,Reclamaзгo de intervalo excessivo da linha,FINALIZADA,NaN,NaN,NaN,Transporte,2,NaN,SPTRANS,NaN,NaN
2,Buraco e pavimentaзгo,PORTAL ONLINE,2017-07-07T00:00:00,2017-09-23T00:00:00,CANGAIBA,2017-11-01T12:48:00,2017-11-01T17:36:00,PENHA,Tapa-buraco,FINALIZADA,NaN,NaN,NaN,Rua e bairro,3,NaN,SMPR,NaN,NaN
3,Criaзгo inadequada de animais,CENTRAL TELEFФNICA,2017-07-07T00:00:00,2017-07-13T00:00:00,VILA PRUDENTE,2017-11-01T12:50:00,2017-11-01T18:45:00,VILA PRUDENTE,Denъncia de condiзхes inadequadas de criaзгo,FINALIZADA,NaN,NaN,NaN,Animais,4,NaN,COVISA-SUVIS,NaN,NaN
4,Drenagem de бgua de chuva,CENTRAL TELEFФNICA,2017-07-07T00:00:00,None,BRASILANDIA,2017-11-01T12:48:00,None,FREGUESIA-BRASILANDIA,"Limpeza de valas, valetas e sarjetхes",AGUARDANDO ATENDIMENTO,NaN,NaN,NaN,Rua e bairro,5,NaN,SMPR,NaN,NaN


In [70]:
# Dados estão com encoding="latin-1"

In [71]:
# Dados de localização dos postes

lampposts_location_id = '441a59c4-44d7-4428-8918-0560fdf145c8'
df_lampposts_loc = pd.DataFrame(get_public_data_sp(lampposts_location_id))
df_lampposts_loc.head()

Getting data from  http://dados.prefeitura.sp.gov.br/api/action/datastore_search?resource_id=441a59c4-44d7-4428-8918-0560fdf145c8


,ID,LAT,LOCAL,LONG,_id
0,1313,-23.5058,parelheiros,-46.4145,1
1,5299,-23.4918,parelheiros,-46.458,2
2,10601,-23.4321,cidade_ademar,-46.3826,3
3,11213,-23.5058,parelheiros,-46.4146,4
4,11820,-23.4128,MBoiMirim,-46.4721,5


In [78]:
# Locais no dataset de atendimentos
df_atends['Prefeitura Regional'].value_counts()

SE                           38
                             31
MOOCA                        23
LAPA                         23
SANTO AMARO                  18
M'BOI MIRIM                  17
PENHA                        17
ITAQUERA                     16
SAO MIGUEL                   15
IPIRANGA                     15
VILA MARIA-VILA GUILHERME    14
BUTANTA                      14
PINHEIROS                    13
CAPELA DO SOCORRO            13
CASA VERDE-CACHOEIRINHA      13
FREGUESIA-BRASILANDIA        12
VILA PRUDENTE                11
CIDADE ADEMAR                11
JABAQUARA                    11
SANTANA-TUCURUVI             10
SAO MATEUS                    9
ARICANDUVA-FORMOSA-CARRAO     8
JACANA-TREMEMBE               8
VILA MARIANA                  8
CAMPO LIMPO                   7
PIRITUBA-JARAGUA              6
SAPOPEMBA                     6
ERMELINO MATARAZZO            3
ITAIM PAULISTA                3
GUAIANASES                    3
PARELHEIROS                   3
CIDADE T

In [74]:
# Locais no dataset de postes
df_lampposts_loc['LOCAL'].value_counts()

parelheiros           16
MBoiMirim             14
Butanta               13
campo_limpo           11
penha                 10
SaoMateus              8
pinheiros              8
Pontos Novos           5
Ermelino_Matarazzo     3
vila_mariana           3
jabaquara              3
capela_socorro         3
ItaimPaulista          1
cidade_ademar          1
pitituba               1
Name: LOCAL, dtype: int64

## Problem on keys

There is no key that can merge the two datasets, so it is a huge risk to take to relate two datasets

In [80]:
# https://developers.google.com/maps/documentation/geocoding/intro
geocode_endpoint = 'https://maps.googleapis.com/maps/api/geocode/json?'

## Trabalhando com dados de BO

In [97]:
# Dados de BO Roubo de Celular
filename = '../data/DadosBO_2017_1(FURTO DE CELULAR).csv'
df_BOs = pd.read_csv(filename, sep=';', encoding='latin-1')
df_BOs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20189 entries, 0 to 20188
Data columns (total 52 columns):
ANO_BO                      20189 non-null int64
NUM_BO                      20189 non-null int64
NUMERO_BOLETIM              20189 non-null object
BO_INICIADO                 20189 non-null object
BO_EMITIDO                  20189 non-null object
DATAOCORRENCIA              20189 non-null object
PERIDOOCORRENCIA            20189 non-null object
DATACOMUNICACAO             20189 non-null object
DATAELABORACAO              20189 non-null object
BO_AUTORIA                  20189 non-null object
FLAGRANTE                   20189 non-null object
NUMERO_BOLETIM_PRINCIPAL    845 non-null object
LOGRADOURO                  16996 non-null object
NUMERO                      20175 non-null float64
BAIRRO                      13889 non-null object
CIDADE                      16996 non-null object
UF                          16996 non-null object
LATITUDE                    16989 non-null o